### **Importing Libs**

In [1]:
!pip install tensorflow-addons
!pip install git+https://github.com/tensorflow/examples.git
!pip install keras

     |████████████████████████████████| 679 kB 4.9 MB/s 
  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-nk284iu7
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-req-build-nk284iu7
  Created wheel for tensorflow-examples: filename=tensorflow_examples-eb9e4340b6d8033a1058c153a3a92f3d328f40e9_-py3-none-any.whl size=270921 sha256=165b2ed94d8e2ecfc4e1b0218044b0301f74d0364d3ede7e659bbc21a0b6f91b
  Stored in directory: /tmp/pip-ephem-wheel-cache-7wh9c40d/wheels/eb/19/50/2a4363c831fa12b400af86325a6f26ade5d2cdc5b406d552ca
Failed to build tensorflow-examples
    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [2]:
from glob import glob
import os
from tensorflow_examples.models.pix2pix import pix2pix
from PIL import Image
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import tensorflow_addons as tfa

from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image

In [3]:
# For more information about autotune:
# https://www.tensorflow.org/guide/data_performance#prefetching
AUTOTUNE = tf.data.experimental.AUTOTUNE
print(f"Tensorflow ver. {tf.__version__}")

Tensorflow ver. 2.5.0


### **Importing Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Path into dataset

dataset_path = "/content/drive/MyDrive/glaucoma_research/RIM-ONE-r3/dataset_v2"
healthy_images = dataset_path + "/healthy/stereo"
glaucoma_images = dataset_path + "/glaucoma_suspect/stereo"
healthy_masked = dataset_path + "/healthy/average_mask"
glaucoma_masked = dataset_path + "/glaucoma_suspect/avg_mask"


In [ ]:
healthy_images

'/content/drive/MyDrive/glaucoma_research/RIM-ONE-r3/dataset_v2/healthy/stereo'

In [ ]:
len(healthy_images)

78

In [ ]:
### Knowing the len of the images
stereo_healthy = len(healthy_images + "*.jpg")
print(f"The originally healthy stereo {stereo_healthy} images.")

masked_healthy = len(healthy_masked + "*png")
print(f"The masked healthy {masked_healthy} images.")

stereo_glaucoma = len(glaucoma_images + "*.jpg")
print(f"The originally glaucoma stereo {stereo_glaucoma} images.")

masked_glaucoma = len(glaucoma_masked + "*.png")
print(f"The masked glaucoma {masked_glaucoma} images.")

The originally healthy stereo 83 images.
The masked healthy 88 images.
The originally glaucoma stereo 92 images.
The masked glaucoma 94 images.


In [ ]:
width = 256
height = 256

In [ ]:
# originally image from the RIM ONe
img_stereo = []
#image masked average considered as the label
label_masked = []
#stereo and masked image as an array if needed
stereo_array = []
masked_array = []

In [ ]:
for i in healthy_images:
  img = image.load_img(healthy_images , ".jpg")
  print(img)
  img = img.resize(width, height)
  image_stereo.append(img)
  img = np.array(img)
  stereo_array(img)

IsADirectoryError: ignored

### **U-Net Implementation**
implementasi u-net yang digunakan merupakan u-net versi pada umumnya

In [ ]:
# -- U-Net Implementation -- #


# -- Encoder -- #
# Block encoder 1

# --- start with uncommend the input and the output --- #
# inputs = Input(shape=input_size)
conv_enc_1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer=initializer)(inputs)
conv_enc_1 = Conv2D(64, 3, activation = 'relu', padding='same', kernel_initializer=initializer)(conv_enc_1)

# Block encoder 2
max_pool_enc_2 = MaxPooling2D(pool_size=(2, 2))(conv_enc_1)
conv_enc_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(max_pool_enc_2)
conv_enc_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_enc_2)

# Block encoder 3
max_pool_enc_3 = MaxPooling2D(pool_size=(2,2))(conv_enc_2)
conv_enc_3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=initializer)(max_pool_enc_3)
conv_enc_3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer=initializer)(conv_enc_3)

# Block  encoder 4
max_pool_enc_4 = MaxPooling2D(pool_size=(2, 2))(conv_enc_3)
conv_enc_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(max_pool_enc_4)
conv_enc_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_enc_4)
# -- Encoder -- #

# ==== #
maxpool = MaxPooling2D(pool_size=(2, 2))(conv_enc_4)
conv = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(maxpool)
conv = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv)
# ==== #

# -- Dencode -- #
# Block decoder 1
up_dec_1 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv))
merge_dec_1 = concatenate([conv_enc_4, up_dec_1], axis = 3)
conv_dec_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_1)
conv_dec_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_1)

# Block decoder 2
up_dec_2 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_1))
merge_dec_2 = concatenate([conv_enc_3, up_dec_2], axis = 3)
conv_dec_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_2)
conv_dec_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_2)

# Block decoder 3
up_dec_3 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_2))
merge_dec_3 = concatenate([conv_enc_2, up_dec_3], axis = 3)
conv_dec_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_3)
conv_dec_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_3)

# Block decoder 4
up_dec_4 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = initializer)(UpSampling2D(size = (2,2))(conv_dec_3))
merge_dec_4 = concatenate([conv_enc_1, up_dec_4], axis = 3)
conv_dec_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(merge_dec_4)
conv_dec_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_4)
conv_dec_4 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = initializer)(conv_dec_4)
# -- Dencoder -- #

#if start uncommend the output too
# output = Conv2D(N_CLASSES, 1, activation = 'sigmoid')(conv_dec_4)

### **Dataset Cite**

@inproceedings{
  inproceedings,
    author = {Fumero, Francisco and Sigut, Jose and Alayón, Silvia  andGonzález-Hernández, M and González de la Rosa, M},
    year = {2015},
    month = {06},
    pages = {},
    title = {Interactive Tool and Database for Optic Disc and   CupSegmentation of Stereo and Monocular Retinal Fundus Images}
}